### Multivariate Analysis

In [1]:
import pandas as pd
from fcns import *

In [2]:
eustocks_df = pd.read_csv('datasets/eustocks.csv')
# freq = 'B' means 5 day week, no holidays
eustocks_df = pd.DataFrame(eustocks_df.values, columns=['0', 'DAX', 'SMI', 'CAC', 'FTSE'], index=pd.DatetimeIndex(start='1991-01-01', periods=1860, freq='B')).drop(columns=['0'])
#print(list(eustocks_df.columns))
eustocks_df.head()


C:\Users\darrahts\Anaconda3\envs\tfgpu\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Creating a DatetimeIndex by passing range endpoints is deprecated.  Use `pandas.date_range` instead.
  This is separate from the ipykernel package so we can avoid doing imports until


,DAX,SMI,CAC,FTSE
1991-01-01,1628.75,1678.1,1772.8,2443.6
1991-01-02,1613.63,1688.5,1750.5,2460.2
1991-01-03,1606.51,1678.6,1718.0,2448.2
1991-01-04,1621.04,1684.1,1708.1,2470.4
1991-01-07,1618.16,1686.6,1723.1,2484.7


In [3]:
import functools
res = eustocks_df.apply(functools.partial(stationarity_test))
res

statistic                  2.295811
p-value                    0.998953
#lags                     24.000000
#obs                    1835.000000
critical value (1%)       -3.433919
critical value (5%)       -2.863116
critical value (10%)      -2.567609
dtype: float64
statistic                  2.235032
p-value                    0.998910
#lags                     22.000000
#obs                    1837.000000
critical value (1%)       -3.433915
critical value (5%)       -2.863115
critical value (10%)      -2.567608
dtype: float64
statistic                  1.286413
p-value                    0.996532
#lags                     17.000000
#obs                    1842.000000
critical value (1%)       -3.433905
critical value (5%)       -2.863110
critical value (10%)      -2.567606
dtype: float64
statistic                  0.106116
p-value                    0.966476
#lags                     13.000000
#obs                    1846.000000
critical value (1%)       -3.433897
critical value (5%)

DAX     0
SMI     0
CAC     0
FTSE    0
dtype: int64

### since non significant, difference the columns

In [4]:
eustocks_df = eustocks_df.diff().dropna()
eustocks_df.head()
res = eustocks_df.apply(functools.partial(stationarity_test))
res

statistic              -8.293747e+00
p-value                 4.193799e-13
#lags                   1.900000e+01
#obs                    1.839000e+03
critical value (1%)    -3.433911e+00
critical value (5%)    -2.863113e+00
critical value (10%)   -2.567607e+00
dtype: float64
statistic              -8.543303e+00
p-value                 9.652044e-14
#lags                   1.900000e+01
#obs                    1.839000e+03
critical value (1%)    -3.433911e+00
critical value (5%)    -2.863113e+00
critical value (10%)   -2.567607e+00
dtype: float64
statistic              -1.015705e+01
p-value                 7.680823e-18
#lags                   1.600000e+01
#obs                    1.842000e+03
critical value (1%)    -3.433905e+00
critical value (5%)    -2.863110e+00
critical value (10%)   -2.567606e+00
dtype: float64
statistic              -1.079549e+01
p-value                 2.080690e-19
#lags                   1.200000e+01
#obs                    1.846000e+03
critical value (1%)    -3.4338

DAX     1
SMI     1
CAC     1
FTSE    1
dtype: int64

### Identify the lag order
- statsmodels.tsa.vector_ar.var_model
- model.select_order
- model.fit(maxlags=X, ic='YY')

In [5]:
from statsmodels.tsa.vector_ar.var_model import VAR

In [6]:
model = VAR(eustocks_df)
best = model.select_order(15)
best.summary()

,AIC,BIC,FPE,HQIC
0,25.31,25.32*,9.833e+10,25.32
1,25.28,25.34,9.505e+10,25.30*
2,25.28,25.39,9.553e+10,25.32
3,25.27,25.43,9.458e+10,25.33
4,25.26,25.47,9.382e+10,25.34
5,25.26,25.51,9.357e+10,25.35
6,25.26,25.56,9.332e+10,25.37
7,25.26,25.61,9.361e+10,25.39
8,25.26,25.66,9.371e+10,25.41
9,25.26,25.70,9.314e+10,25.42


In [7]:
res = model.fit(maxlags=15, ic='aic')
res.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Mon, 04, Nov, 2019
Time:                     10:45:31
--------------------------------------------------------------------
No. of Equations:         4.00000    BIC:                    25.7862
Nobs:                     1848.00    HQIC:                   25.4466
Log likelihood:          -33638.3    FPE:                9.23092e+10
AIC:                      25.2484    Det(Omega_mle):     8.38399e+10
--------------------------------------------------------------------
Results for equation DAX
              coefficient       std. error           t-stat            prob
---------------------------------------------------------------------------
const            1.689997         0.772454            2.188           0.029
L1.DAX          -0.002794         0.042406           -0.066           0.947
L1.SMI          -0.106047         0.029764           -3.563           0.000
L1.CAC

### Granger causality test
 - https://towardsdatascience.com/granger-causality-and-vector-auto-regressive-model-for-time-series-forecasting-3226a64889a6

In [8]:
granger = res.test_causality(['SMI', 'CAC', 'FTSE'], ['DAX'], kind='f')
granger.summary()

Test statistic,Critical value,p-value,df
1.681,1.438,0.009,"(33, 7212)"
